## Yolo aproach on MiTec dataset

In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch import nn
from torch.optim import Adam
from torchvision.transforms import  Compose, ToTensor, Resize, RandomHorizontalFlip, RandomRotation, ColorJitter, RandomResizedCrop, Lambda
from PIL import Image
import matplotlib.pyplot as plt
from torchsummary import summary
import torchmetrics
from lightning.pytorch.loggers import WandbLogger
import wandb
import random
from datetime import datetime
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
%matplotlib inline 

In [3]:
import os

# Model
%env CUDA_VISIBLE_DEVICES=2
#%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:32

# HF Cache
os.environ["HF_HOME"] = "../.cache"
!echo $HF_HOME
!huggingface-cli whoami

%matplotlib inline 


env: CUDA_VISIBLE_DEVICES=2
../.cache
Not logged in


In [7]:
import os
import shutil

def get_mvtec_with_classes(class_list, image_path, annotation_path, output_path, name="test_set", info=True, map_ids=None):
    valid_images = []

    if not info:
        # Zielordnerstruktur erstellen
        images_out = os.path.join(output_path, "images", name)
        labels_out = os.path.join(output_path, "labels", name)

        os.makedirs(images_out, exist_ok=True)
        os.makedirs(labels_out, exist_ok=True)

    for subfolder in os.listdir(annotation_path):
        folder_path = os.path.join(annotation_path, subfolder)
        valid_count = 0

        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)

            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()

            converted_lines = []
            all_valid = True

            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])

                if class_id not in class_list:
                    all_valid = False
                    break

                # Mappe die ID falls map_ids gegeben ist
                new_class_id = map_ids[class_id] if map_ids and class_id in map_ids else class_id
                converted_line = f"{new_class_id} " + " ".join(parts[1:])
                converted_lines.append(converted_line)

            if all_valid:
                image_filename = os.path.splitext(file)[0] + ".jpg"
                image_full_path = os.path.join(image_path, subfolder, image_filename)

                if os.path.exists(image_full_path):
                    valid_images.append(image_full_path)
                    valid_count += 1

                    if not info:
                        os.makedirs(images_out, exist_ok=True)
                        os.makedirs(labels_out, exist_ok=True)

                        # Bild kopieren
                        shutil.copy2(image_full_path, os.path.join(images_out, image_filename))

                        # Label mit umgeschriebener ID speichern
                        label_target_path = os.path.join(labels_out, file)
                        with open(label_target_path, "w", encoding="utf-8") as f:
                            f.write("\n".join(converted_lines) + "\n")

        if info:
            print(f"{subfolder}: {valid_count} gültige Bilder")

    return valid_images


In [8]:
map_ids = {25: 0, 26: 0, 27: 0, 28: 1, 50: 4, 51: 3, 29: 2, 30: 2, 31: 5, 32: 5}

get_mvtec_with_classes(
    [25, 26, 27, 28, 50, 51, 29, 30, 31, 32],
    "../../huggingface/mvtec_annotated/images",
    "../../huggingface/mvtec_annotated/labels",
    "../../huggingface/ai_shelf/artificial_created_dataset",
    name="mv_tec_images",
    info=False,
    map_ids=map_ids
)


['../../huggingface/mvtec_annotated/images/train/D2S_009403.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_016124.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_022905.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_018321.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_003611.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_013815.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_008827.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_009405.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_012516.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_018311.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_018223.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_022903.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_018310.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_009303.jpg',
 '../../huggingface/mvtec_annotated/images/train/D2S_008706.jpg',
 '../../hu

In [ ]:
def configure_wandb(additional_name="", tags=[]):
    # Sort the dataset names aplhabetically
    dataset_names = "-".join(sorted([os.path.basename(path) for path in DATASET_PATHS]))
    group = "Regression-"+dataset_names
    name = dataset_names+additional_name+"_"+datetime.now().strftime("%d%b-%H:%M:%S")

    wandb_logger = WandbLogger(project="TinyML-CartDetection", group=group, name=name, tags=tags)   
    wandb_logger.experiment.config.update({"batch_size": BATCH_SIZE, "image_size": IMAGE_SIZE, "seed": SEED}) # Log additional hyperparameters

    return wandb_logger

In [14]:
from ultralytics import YOLO
import wandb
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import math


test_image_dir = Path("../../huggingface/ai_shelf/artificial_created_dataset/images/mv_tec_images")
test_images = list(test_image_dir.glob("*.jpg"))[:20]  # Nur die ersten 20
print(len(test_images))
# 1. Login bei Weights & Biases (W&B)
# wandb.login()

# # 2. Initialisiere W&B Run (optional)
# wandb.init(
#     project="Yolo-Training",
#     entity="maats",
#     name="YOLO11n-COCO11-testing_with_mv_tec_train_on_generated",
#     config={  # alle Hyperparameter sauber abspeichern
#         "epochs": 20,
#         "imgsz": 400,
#         "model": "yolo11n.pt",
#         "dataset": "atrifitial_created_dataset.yaml",
#     },
#     sync_tensorboard=True,   # (Optional) falls du TensorBoard benutzt
# )




try:
    # 3. YOLO-Modell laden (z.B. yolo11n)
    model = YOLO("yolo11n.pt")

    # 4. Trainiere das Modell
    # 4. Trainiere das Modell
    results = model.train(
        data="/data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/artificial_created_dataset/atrifitial_created_dataset.yaml",
        epochs=5,
        imgsz=400,
        project="runs/train",
        name="YOLO11n-COCO11-testing_with_mv_tec_train_on_generated",
        verbose=True,
        # 🛠 WICHTIG:
        val=True,       # Validierung nach jedem Epoch aktivieren
        save=True,      # Modelle speichern
        save_period=1,  # (Optional) jedes Epoch speichern
        mode="wandb"    # ✅ AUTOMATISCH alle YOLO Metriken an W&B schicken!
    )
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # wandb.finish()
    print("done")

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
import os

# Sicherstellen, dass headless läuft
matplotlib.use("Agg")

n_cols = 3
n_rows = math.ceil(len(test_images) / n_cols)

fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
axs = axs.flatten()

# Speicherpfad für Ergebnisse
output_dir = Path("predicted_results")
output_dir.mkdir(exist_ok=True)

for idx, img_path in enumerate(test_images):
    results = model(img_path)

    img = cv2.imread(str(img_path))
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            conf = box.conf[0]
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # RGB-Konvertierung für Matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Einzelbild speichern
    out_path = output_dir / f"{img_path.stem}_pred.png"
    cv2.imwrite(str(out_path), img)

    # Anzeige vorbereiten
    axs[idx].imshow(img_rgb)
    axs[idx].axis('off')
    axs[idx].set_title(img_path.name)

# Leere Felder im Grid ausblenden
for j in range(idx + 1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()

# Komplettes Grid speichern
grid_path = output_dir / "prediction_grid.png"
fig.savefig(grid_path)
print(f"Gespeichertes Grid: {grid_path}")

# Versuch, im Notebook anzuzeigen
try:
    display(fig)
except Exception as e:
    print(f"Anzeige im Notebook fehlgeschlagen: {e}")




20
Ultralytics 8.3.129 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA TITAN Xp, 12183MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/artificial_created_dataset/atrifitial_created_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=400, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLO11n-COCO11-testing_with_mv_tec_train_on_generated5, 

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


TensorBoard: Start with 'tensorboard --logdir runs/train/YOLO11n-COCO11-testing_with_mv_tec_train_on_generated5', view at http://localhost:6006/
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2272.1±483.2 MB/s, size: 142.7 KB)


train: Scanning /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/artificial_created_dataset/labels/train.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 877.2±617.7 MB/s, size: 142.2 KB)


...

                   all        300       1410      0.986      0.987      0.989      0.774



5 epochs completed in 0.016 hours.
Optimizer stripped from runs/train/YOLO11n-COCO11-testing_with_mv_tec_train_on_generated5/weights/last.pt, 5.4MB
Optimizer stripped from runs/train/YOLO11n-COCO11-testing_with_mv_tec_train_on_generated5/weights/best.pt, 5.4MB

Validating runs/train/YOLO11n-COCO11-testing_with_mv_tec_train_on_generated5/weights/best.pt...
Ultralytics 8.3.129 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA TITAN Xp, 12183MiB)
YOLO11n summary (fused): 100 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  3.33it/s]


                   all        300       1410      0.986      0.987      0.989      0.774
                 apple        152        217          1      0.993      0.995       0.85
               avocado        160        228          1      0.996      0.995      0.751
                banana        122        155      0.974      0.981      0.988      0.571
              cucumber         98        114       0.95      0.947      0.965      0.709
             fruit tea        126        174      0.992          1      0.995      0.855
                 lemon        185        309      0.999          1      0.995      0.765
          tomato sauce        152        213      0.987      0.995       0.99      0.915
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/train/YOLO11n-COCO11-testing_with_mv_tec_train_on_generated5
done

image 1/1 /data22/stu236894/GitRepos/TinyML-MT/code_training/classification-code/../../huggingface/ai_shelf/artificia

<Figure size 1500x3500 with 20 Axes>